In [ ]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [ ]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [ ]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets3.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "zv7uwCjVmJpYMtPgUmXC5dZPa"
    consumer_secret = "ZVK8XYIAG4yI15Sbl4tL2SNYrKmEAvTMyj0OWvm1myI1n6yizA"
    access_token = "1517271973868437506-iHXdg3cpAV6l9qSfu79pcasEVrMwa7"
    access_token_secret = "7b4t3VBJX4HnUhTrJhD4EyInvatoVwxJamhIr1OPzEvJN"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-122.769302, 33.038687, -117.649673, 38.154078]  # Thebay


    stream_listener = StreamListener(time_limit=300, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS, languages=['en'], encoding="utf-8")

{'id': 1518377614557077504, 'username': 'AshockiMizer5', 'created_at': 'Sun Apr 24 23:53:28 +0000 2022', 'lng': -121.8737305, 'lat': 37.331159, 'text': '@AyoTBailey4 @DailyLoonaHB @PatrickHZala It IS NOT REAL STOP SAYING THAT OVER AND OVER AGAIN!!!!!!!!!!!!!! https://t.co/Qbq0BgHsxa'}
{'id': 1518377620710170625, 'username': 'blove416', 'created_at': 'Sun Apr 24 23:53:29 +0000 2022', 'lng': -121.436217, 'lat': 37.723349999999996, 'text': '@Califinest831 @itzTdizzle510 @babiiface831 @jpcrin Toms my guy. Don’t hate'}
{'id': 1518377621544861696, 'username': 'MichaelCEssien', 'created_at': 'Sun Apr 24 23:53:29 +0000 2022', 'lng': -122.2784865, 'lat': 37.758798, 'text': 'That part ⬇️'}
{'id': 1518377627332927488, 'username': 'gmeda72', 'created_at': 'Sun Apr 24 23:53:31 +0000 2022', 'lng': -117.772917, 'lat': 34.0654505, 'text': "@realDailyWire She's been a drag to our nation for several years now!!"}
{'id': 1518377647671156736, 'username': 'ericjunior', 'created_at': 'Sun Apr 24 23:53:35 +0

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
